# Auto Dataset con pytorch

Imos a tratar o automovile dataset con polars

In [20]:
import torch
from torch.utils.tensorboard import SummaryWriter
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda
NVIDIA GeForce RTX 3060
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [21]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt

plt.style.use('ggplot')


# irisDataset = pl.scan_csv("iris.data", has_header=False,with_column_names=lambda cols: ["sepallength","sepalwidth","petallength","petalwidth","class"])

irisDataset = pl.read_csv("imports-85.data", has_header=False,
                          new_columns=["symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors", "body-style",
                          "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type",
                          "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", 
                          "peak-rpm", "city-mpg", "highway-mpg", "price"], 
                          null_values=['?'])

No csv non temos a fila de cabeceira, polo que lla engadimos nos a man

In [22]:
display(irisDataset)
tamanhoV1 = irisDataset.estimated_size()
print(f"Memoria usada: {tamanhoV1}")

symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
i64,i64,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,str,str,i64,str,f64,f64,f64,i64,i64,i64,i64,i64
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.6,168.8,64.1,48.8,2548,"""dohc""","""four""",130,"""mpfi""",3.47,2.68,9.0,111,5000,21,27,13495
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.6,168.8,64.1,48.8,2548,"""dohc""","""four""",130,"""mpfi""",3.47,2.68,9.0,111,5000,21,27,16500
1,null,"""alfa-romero""","""gas""","""std""","""two""","""hatchback""","""rwd""","""front""",94.5,171.2,65.5,52.4,2823,"""ohcv""","""six""",152,"""mpfi""",2.68,3.47,9.0,154,5000,19,26,16500
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""",99.8,176.6,66.2,54.3,2337,"""ohc""","""four""",109,"""mpfi""",3.19,3.4,10.0,102,5500,24,30,13950
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""4wd""","""front""",99.4,176.6,66.4,54.3,2824,"""ohc""","""five""",136,"""mpfi""",3.19,3.4,8.0,115,5500,18,22,17450
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-1,95,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.1,188.8,68.9,55.5,2952,"""ohc""","""four""",141,"""mpfi""",3.78,3.15,9.5,114,5400,23,28,16845
-1,95,"""volvo""","""gas""","""turbo""","""four""","""sedan""","""rwd""","""front""",109.1,188.8,68.8,55.5,3049,"""ohc""","""four""",141,"""mpfi""",3.78,3.15,8.7,160,5300,19,25,19045
-1,95,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.1,188.8,68.9,55.5,3012,"""ohcv""","""six""",173,"""mpfi""",3.58,2.87,8.8,134,5500,18,23,21485


Memoria usada: 59326


In [23]:
irisDataset.head()

symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
i64,i64,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,str,str,i64,str,f64,f64,f64,i64,i64,i64,i64,i64
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.6,168.8,64.1,48.8,2548,"""dohc""","""four""",130,"""mpfi""",3.47,2.68,9.0,111,5000,21,27,13495
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.6,168.8,64.1,48.8,2548,"""dohc""","""four""",130,"""mpfi""",3.47,2.68,9.0,111,5000,21,27,16500
1,null,"""alfa-romero""","""gas""","""std""","""two""","""hatchback""","""rwd""","""front""",94.5,171.2,65.5,52.4,2823,"""ohcv""","""six""",152,"""mpfi""",2.68,3.47,9.0,154,5000,19,26,16500
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""",99.8,176.6,66.2,54.3,2337,"""ohc""","""four""",109,"""mpfi""",3.19,3.4,10.0,102,5500,24,30,13950
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""4wd""","""front""",99.4,176.6,66.4,54.3,2824,"""ohc""","""five""",136,"""mpfi""",3.19,3.4,8.0,115,5500,18,22,17450


In [24]:
irisDataset.describe()

statistic,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
str,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",205.0,164.0,"""205""","""205""","""205""","""203""","""205""","""205""","""205""",205.0,205.0,205.0,205.0,205.0,"""205""","""205""",205.0,"""205""",201.0,201.0,205.0,203.0,203.0,205.0,205.0,201.0
"""null_count""",0.0,41.0,"""0""","""0""","""0""","""2""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""",0.0,"""0""",4.0,4.0,0.0,2.0,2.0,0.0,0.0,4.0
"""mean""",0.834146,122.0,null,null,null,null,null,null,null,98.756585,174.049268,65.907805,53.724878,2555.565854,null,null,126.907317,null,3.329751,3.255423,10.142537,104.256158,5125.369458,25.219512,30.75122,13207.129353
"""std""",1.245307,35.442168,null,null,null,null,null,null,null,6.021776,12.337289,2.145204,2.443522,520.680204,null,null,41.642693,null,0.273539,0.316717,3.97204,39.714369,479.33456,6.542142,6.886443,7947.066342
"""min""",-2.0,65.0,"""alfa-romero""","""diesel""","""std""","""four""","""convertible""","""4wd""","""front""",86.6,141.1,60.3,47.8,1488.0,"""dohc""","""eight""",61.0,"""1bbl""",2.54,2.07,7.0,48.0,4150.0,13.0,16.0,5118.0
"""25%""",0.0,94.0,null,null,null,null,null,null,null,94.5,166.3,64.1,52.0,2145.0,null,null,97.0,null,3.15,3.11,8.6,70.0,4800.0,19.0,25.0,7775.0
"""50%""",1.0,115.0,null,null,null,null,null,null,null,97.0,173.2,65.5,54.1,2414.0,null,null,120.0,null,3.31,3.29,9.0,95.0,5200.0,24.0,30.0,10295.0
"""75%""",2.0,150.0,null,null,null,null,null,null,null,102.4,183.1,66.9,55.5,2935.0,null,null,141.0,null,3.59,3.41,9.4,116.0,5500.0,30.0,34.0,16500.0
"""max""",3.0,256.0,"""volvo""","""gas""","""turbo""","""two""","""wagon""","""rwd""","""rear""",120.9,208.1,72.3,59.8,4066.0,"""rotor""","""two""",326.0,"""spfi""",3.94,4.17,23.0,288.0,6600.0,49.0,54.0,45400.0


In [25]:
irisDatasetv2 = pl.read_csv("imports-85.data", dtypes={
            "column_1": pl.Int8,
            "column_2": pl.Int16,
            "column_3": pl.String,
            "column_4": pl.String,
            "column_5": pl.String,
            "column_6": pl.String,
            "column_7": pl.String,
            "column_8": pl.String,
            "column_9": pl.String,
            "column_10": pl.Float32,
            "column_11": pl.Float32,
            "column_12": pl.Float32,
            "column_13": pl.Float32,
            "column_14": pl.Float32,
            "column_15": pl.String,
            "column_16": pl.String,
            "column_17": pl.String,
            "column_18": pl.String,
            "column_19": pl.Float32,
            "column_20": pl.Float32,
            "column_21": pl.Float32,
            "column_22": pl.Float32,
            "column_23": pl.Float32,
            "column_24": pl.Float32,
            "column_25": pl.Float32,
            "column_26": pl.Float32
        },has_header=False,new_columns=["symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors", "body-style",
                          "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type",
                          "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", 
                          "peak-rpm", "city-mpg", "highway-mpg", "price"], 
        null_values=['?'])
display(irisDatasetv2)
tamanhoV2 = irisDatasetv2.estimated_size()

print(f"Memoria usada: {tamanhoV2}, porcentaxe: {tamanhoV2/tamanhoV1 *100:2.2f}%")

symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
i8,i16,str,str,str,str,str,str,str,f32,f32,f32,f32,f32,str,str,str,str,f32,f32,f32,f32,f32,f32,f32,f32
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.599998,168.800003,64.099998,48.799999,2548.0,"""dohc""","""four""","""130""","""mpfi""",3.47,2.68,9.0,111.0,5000.0,21.0,27.0,13495.0
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.599998,168.800003,64.099998,48.799999,2548.0,"""dohc""","""four""","""130""","""mpfi""",3.47,2.68,9.0,111.0,5000.0,21.0,27.0,16500.0
1,null,"""alfa-romero""","""gas""","""std""","""two""","""hatchback""","""rwd""","""front""",94.5,171.199997,65.5,52.400002,2823.0,"""ohcv""","""six""","""152""","""mpfi""",2.68,3.47,9.0,154.0,5000.0,19.0,26.0,16500.0
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""",99.800003,176.600006,66.199997,54.299999,2337.0,"""ohc""","""four""","""109""","""mpfi""",3.19,3.4,10.0,102.0,5500.0,24.0,30.0,13950.0
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""4wd""","""front""",99.400002,176.600006,66.400002,54.299999,2824.0,"""ohc""","""five""","""136""","""mpfi""",3.19,3.4,8.0,115.0,5500.0,18.0,22.0,17450.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-1,95,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.900002,55.5,2952.0,"""ohc""","""four""","""141""","""mpfi""",3.78,3.15,9.5,114.0,5400.0,23.0,28.0,16845.0
-1,95,"""volvo""","""gas""","""turbo""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.800003,55.5,3049.0,"""ohc""","""four""","""141""","""mpfi""",3.78,3.15,8.7,160.0,5300.0,19.0,25.0,19045.0
-1,95,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.900002,55.5,3012.0,"""ohcv""","""six""","""173""","""mpfi""",3.58,2.87,8.8,134.0,5500.0,18.0,23.0,21485.0


Memoria usada: 47641, porcentaxe: 80.30%


In [26]:
irisDatasetv3 = pl.read_csv("imports-85.data", dtypes={
            "column_1": pl.Int8,
            "column_2": pl.Int16,
            "column_3": pl.Categorical,
            "column_4": pl.Categorical,
            "column_5": pl.Categorical,
            "column_6": pl.Categorical,
            "column_7": pl.Categorical,
            "column_8": pl.Categorical,
            "column_9": pl.Categorical,
            "column_10": pl.Float32,
            "column_11": pl.Float32,
            "column_12": pl.Float32,
            "column_13": pl.Float32,
            "column_14": pl.Float32,
            "column_15": pl.Categorical,
            "column_16": pl.Categorical,
            "column_17": pl.Float32,
            "column_18": pl.Categorical,
            "column_19": pl.Float32,
            "column_20": pl.Float32,
            "column_21": pl.Float32,
            "column_22": pl.Float32,
            "column_23": pl.Float32,
            "column_24": pl.Float32,
            "column_25": pl.Float32,
            "column_26": pl.Float32
        },has_header=False,new_columns=["symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors", "body-style",
                          "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type",
                          "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", 
                          "peak-rpm", "city-mpg", "highway-mpg", "price"], 
        null_values=['?']) #.drop_nulls()
display(irisDatasetv3)
tamanhoV3 = irisDatasetv3.estimated_size()

print(f"Memoria usada: {tamanhoV3}, porcentaxe: {tamanhoV3/tamanhoV1 *100:2.2f}%")

symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
i8,i16,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,cat,cat,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.599998,168.800003,64.099998,48.799999,2548.0,"""dohc""","""four""",130.0,"""mpfi""",3.47,2.68,9.0,111.0,5000.0,21.0,27.0,13495.0
3,null,"""alfa-romero""","""gas""","""std""","""two""","""convertible""","""rwd""","""front""",88.599998,168.800003,64.099998,48.799999,2548.0,"""dohc""","""four""",130.0,"""mpfi""",3.47,2.68,9.0,111.0,5000.0,21.0,27.0,16500.0
1,null,"""alfa-romero""","""gas""","""std""","""two""","""hatchback""","""rwd""","""front""",94.5,171.199997,65.5,52.400002,2823.0,"""ohcv""","""six""",152.0,"""mpfi""",2.68,3.47,9.0,154.0,5000.0,19.0,26.0,16500.0
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""",99.800003,176.600006,66.199997,54.299999,2337.0,"""ohc""","""four""",109.0,"""mpfi""",3.19,3.4,10.0,102.0,5500.0,24.0,30.0,13950.0
2,164,"""audi""","""gas""","""std""","""four""","""sedan""","""4wd""","""front""",99.400002,176.600006,66.400002,54.299999,2824.0,"""ohc""","""five""",136.0,"""mpfi""",3.19,3.4,8.0,115.0,5500.0,18.0,22.0,17450.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-1,95,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.900002,55.5,2952.0,"""ohc""","""four""",141.0,"""mpfi""",3.78,3.15,9.5,114.0,5400.0,23.0,28.0,16845.0
-1,95,"""volvo""","""gas""","""turbo""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.800003,55.5,3049.0,"""ohc""","""four""",141.0,"""mpfi""",3.78,3.15,8.7,160.0,5300.0,19.0,25.0,19045.0
-1,95,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.900002,55.5,3012.0,"""ohcv""","""six""",173.0,"""mpfi""",3.58,2.87,8.8,134.0,5500.0,18.0,23.0,21485.0


Memoria usada: 22074, porcentaxe: 37.21%


In [27]:
irisDatasetv4 = pl.read_csv("imports-85.data", dtypes={
            "column_1": pl.Int8,
            "column_2": pl.Float32,
            "column_3": pl.Categorical,
            "column_4": pl.Categorical,
            "column_5": pl.Categorical,
            "column_6": pl.Categorical,
            "column_7": pl.Categorical,
            "column_8": pl.Categorical,
            "column_9": pl.Categorical,
            "column_10": pl.Float32,
            "column_11": pl.Float32,
            "column_12": pl.Float32,
            "column_13": pl.Float32,
            "column_14": pl.Float32,
            "column_15": pl.Categorical,
            "column_16": pl.Categorical,
            "column_17": pl.Float32,
            "column_18": pl.Categorical,
            "column_19": pl.Float32,
            "column_20": pl.Float32,
            "column_21": pl.Float32,
            "column_22": pl.Float32,
            "column_23": pl.Float32,
            "column_24": pl.Float32,
            "column_25": pl.Float32,
            "column_26": pl.Float32
        },has_header=False,new_columns=["symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors", "body-style",
                          "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type",
                          "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", 
                          "peak-rpm", "city-mpg", "highway-mpg", "price"], 
        null_values=['?']).drop_nulls()
display(irisDatasetv4)
tamanhoV4 = irisDatasetv4.estimated_size()

print(f"Memoria usada: {tamanhoV4}, porcentaxe: {tamanhoV4/tamanhoV1 *100:2.2f}%")

symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
i8,f32,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,cat,cat,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32
2,164.0,"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""",99.800003,176.600006,66.199997,54.299999,2337.0,"""ohc""","""four""",109.0,"""mpfi""",3.19,3.4,10.0,102.0,5500.0,24.0,30.0,13950.0
2,164.0,"""audi""","""gas""","""std""","""four""","""sedan""","""4wd""","""front""",99.400002,176.600006,66.400002,54.299999,2824.0,"""ohc""","""five""",136.0,"""mpfi""",3.19,3.4,8.0,115.0,5500.0,18.0,22.0,17450.0
1,158.0,"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""",105.800003,192.699997,71.400002,55.700001,2844.0,"""ohc""","""five""",136.0,"""mpfi""",3.19,3.4,8.5,110.0,5500.0,19.0,25.0,17710.0
1,158.0,"""audi""","""gas""","""turbo""","""four""","""sedan""","""fwd""","""front""",105.800003,192.699997,71.400002,55.900002,3086.0,"""ohc""","""five""",131.0,"""mpfi""",3.13,3.4,8.3,140.0,5500.0,17.0,20.0,23875.0
2,192.0,"""bmw""","""gas""","""std""","""two""","""sedan""","""rwd""","""front""",101.199997,176.800003,64.800003,54.299999,2395.0,"""ohc""","""four""",108.0,"""mpfi""",3.5,2.8,8.8,101.0,5800.0,23.0,29.0,16430.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-1,95.0,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.900002,55.5,2952.0,"""ohc""","""four""",141.0,"""mpfi""",3.78,3.15,9.5,114.0,5400.0,23.0,28.0,16845.0
-1,95.0,"""volvo""","""gas""","""turbo""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.800003,55.5,3049.0,"""ohc""","""four""",141.0,"""mpfi""",3.78,3.15,8.7,160.0,5300.0,19.0,25.0,19045.0
-1,95.0,"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""",109.099998,188.800003,68.900002,55.5,3012.0,"""ohcv""","""six""",173.0,"""mpfi""",3.58,2.87,8.8,134.0,5500.0,18.0,23.0,21485.0


Memoria usada: 17796, porcentaxe: 30.00%


# Visualizacion

In [28]:
medias = irisDatasetv4.mean()

grupo_numericos = irisDatasetv4.select([pl.col(pl.Int8),pl.col(pl.Int16),pl.col(pl.Float32)])
display(grupo_numericos)
medias_solo = torch.tensor(grupo_numericos.mean().to_numpy()).squeeze()
stds = irisDatasetv4.std()
stds_solo = torch.tensor(grupo_numericos.std().to_numpy()).squeeze()
display(medias)
display(medias_solo)
display(medias_solo.shape)
display(medias_solo.shape)
display(stds)
display(stds_solo)



symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2,164.0,99.800003,176.600006,66.199997,54.299999,2337.0,109.0,3.19,3.4,10.0,102.0,5500.0,24.0,30.0,13950.0
2,164.0,99.400002,176.600006,66.400002,54.299999,2824.0,136.0,3.19,3.4,8.0,115.0,5500.0,18.0,22.0,17450.0
1,158.0,105.800003,192.699997,71.400002,55.700001,2844.0,136.0,3.19,3.4,8.5,110.0,5500.0,19.0,25.0,17710.0
1,158.0,105.800003,192.699997,71.400002,55.900002,3086.0,131.0,3.13,3.4,8.3,140.0,5500.0,17.0,20.0,23875.0
2,192.0,101.199997,176.800003,64.800003,54.299999,2395.0,108.0,3.5,2.8,8.8,101.0,5800.0,23.0,29.0,16430.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-1,95.0,109.099998,188.800003,68.900002,55.5,2952.0,141.0,3.78,3.15,9.5,114.0,5400.0,23.0,28.0,16845.0
-1,95.0,109.099998,188.800003,68.800003,55.5,3049.0,141.0,3.78,3.15,8.7,160.0,5300.0,19.0,25.0,19045.0
-1,95.0,109.099998,188.800003,68.900002,55.5,3012.0,173.0,3.58,2.87,8.8,134.0,5500.0,18.0,23.0,21485.0


symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
f64,f32,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,cat,cat,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32
0.735849,121.132072,null,null,null,null,null,null,null,98.264153,172.413834,65.607544,53.899372,2461.138428,null,null,119.226418,null,3.300126,3.236352,10.161132,95.836479,5113.836426,26.522013,32.08176,11445.729492


tensor([7.3585e-01, 1.2113e+02, 9.8264e+01, 1.7241e+02, 6.5608e+01, 5.3899e+01,
        2.4611e+03, 1.1923e+02, 3.3001e+00, 3.2364e+00, 1.0161e+01, 9.5836e+01,
        5.1138e+03, 2.6522e+01, 3.2082e+01, 1.1446e+04], dtype=torch.float64)

torch.Size([16])

torch.Size([16])

symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
f64,f32,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,cat,cat,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32
1.193086,35.651283,null,null,null,null,null,null,null,5.167417,11.523177,1.947883,2.268761,481.941315,null,null,30.460791,null,0.267336,0.294888,3.889475,30.718582,465.754852,6.097142,6.459189,5877.855957


tensor([1.1931e+00, 3.5651e+01, 5.1674e+00, 1.1523e+01, 1.9479e+00, 2.2688e+00,
        4.8194e+02, 3.0461e+01, 2.6734e-01, 2.9489e-01, 3.8895e+00, 3.0719e+01,
        4.6575e+02, 6.0971e+00, 6.4592e+00, 5.8779e+03], dtype=torch.float64)

In [29]:
# import torch


class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native functions. The module does not expect the
        tensors to be of any specific shape; as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)
        

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)

    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)

    def __call__(self, sample):
        values,saidas = sample
        return ((values - self.mean) / (self.std + self.epsilon), saidas)
        
    def __repr__(self):
        return f"mean: {self.mean}, std:{self.std}, epsilon:{self.epsilon}"

# torch_tensor = torch.tensor(XSinHead.values)
# scaler = StandardScaler()
# scaler.fit(torch_tensor)
# display(scaler)
# XScalada = scaler.fit_transform(torch_tensor)

E agora creamos unha instancia de standar scaler.



In [30]:
scaler = StandardScaler(medias_solo, stds_solo)
display(scaler)

mean: tensor([7.3585e-01, 1.2113e+02, 9.8264e+01, 1.7241e+02, 6.5608e+01, 5.3899e+01,
        2.4611e+03, 1.1923e+02, 3.3001e+00, 3.2364e+00, 1.0161e+01, 9.5836e+01,
        5.1138e+03, 2.6522e+01, 3.2082e+01, 1.1446e+04], dtype=torch.float64), std:tensor([1.1931e+00, 3.5651e+01, 5.1674e+00, 1.1523e+01, 1.9479e+00, 2.2688e+00,
        4.8194e+02, 3.0461e+01, 2.6734e-01, 2.9489e-01, 3.8895e+00, 3.0719e+01,
        4.6575e+02, 6.0971e+00, 6.4592e+00, 5.8779e+03], dtype=torch.float64), epsilon:1e-07

In [31]:
npSample = irisDatasetv4.select([pl.col(pl.Int8),pl.col(pl.Int16),pl.col(pl.Float32)])[0].to_numpy()
tSample = torch.Tensor(npSample)
display(tSample)
sample_escalado = scaler.transform(tSample)
display(sample_escalado)

tensor([[2.0000e+00, 1.6400e+02, 9.9800e+01, 1.7660e+02, 6.6200e+01, 5.4300e+01,
         2.3370e+03, 1.0900e+02, 3.1900e+00, 3.4000e+00, 1.0000e+01, 1.0200e+02,
         5.5000e+03, 2.4000e+01, 3.0000e+01, 1.3950e+04]])

tensor([[ 1.0596,  1.2024,  0.2972,  0.3633,  0.3042,  0.1766, -0.2576, -0.3357,
         -0.4119,  0.5549, -0.0414,  0.2006,  0.8291, -0.4136, -0.3223,  0.4261]],
       dtype=torch.float64)

# One Hot Encoding

Ou como transformar columnas a dummies

Na entrada deste dataset hai moitos categóricos. Aínda que existe a función para facer os dummies, non vai a valer cando se tente facer 
dentro do Dataset de torch, xa que non teremos o dataset completo, senón que so teremos unhas cantas filas. Po riso, aínda que polars ten un método para obter os dummies, non é moi útil neste caso.

Con polars podemos facer a transformación dun xeito eficiente, pero hai que ter en conta todos os valores posibles:
- Se pode obter dun dataset o conxunto de columnas categoricas
- Para cada categoría se pode extraer o conxunto de valores
- Para cada columna e valor posible, se pode facer unha expresión que lle diga a polars:
    - Crea unha nova columna que se chama dun xeito concreto e predecible
    - Se o valor da columna orixinal ten un valor concreto se pon a true a nova columna, senón false

Por exemplo:
```python
expr_list = [
      (pl.col('fuel-type') == i ).alias(f"fuel-type-{i}")
        for i in ['diesel','gas']
    ]
```
Isto aplicado a un dataset:
```python
novos_datos = datos.select([pl.col(pl.Categorical)]).with_columns(
      expr_list
    )
```    

In [32]:
expr_list = [
      (pl.col('fuel-type') == i ).alias(f"fuel-type-{i}")
        for i in ['diesel','gas']
    ]

proba = irisDatasetv4.select([pl.col(pl.Categorical)]).with_columns(
      expr_list
    )
display(proba)

make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,engine-type,num-of-cylinders,fuel-system,fuel-type-diesel,fuel-type-gas
cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,bool,bool
"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""","""ohc""","""four""","""mpfi""",false,true
"""audi""","""gas""","""std""","""four""","""sedan""","""4wd""","""front""","""ohc""","""five""","""mpfi""",false,true
"""audi""","""gas""","""std""","""four""","""sedan""","""fwd""","""front""","""ohc""","""five""","""mpfi""",false,true
"""audi""","""gas""","""turbo""","""four""","""sedan""","""fwd""","""front""","""ohc""","""five""","""mpfi""",false,true
"""bmw""","""gas""","""std""","""two""","""sedan""","""rwd""","""front""","""ohc""","""four""","""mpfi""",false,true
…,…,…,…,…,…,…,…,…,…,…,…
"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""","""ohc""","""four""","""mpfi""",false,true
"""volvo""","""gas""","""turbo""","""four""","""sedan""","""rwd""","""front""","""ohc""","""four""","""mpfi""",false,true
"""volvo""","""gas""","""std""","""four""","""sedan""","""rwd""","""front""","""ohcv""","""six""","""mpfi""",false,true


Podes ver que na saída anterior aparecen doas novas columnas, fuel-type-diesel e fuel-type-gas. 

Faltaría eliminar a columna fuel-type.

In [33]:
proba = irisDatasetv4.select([pl.col(pl.Categorical)]).with_columns(
      expr_list
    ).drop('fuel-type')
display(proba)

make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,engine-type,num-of-cylinders,fuel-system,fuel-type-diesel,fuel-type-gas
cat,cat,cat,cat,cat,cat,cat,cat,cat,bool,bool
"""audi""","""std""","""four""","""sedan""","""fwd""","""front""","""ohc""","""four""","""mpfi""",false,true
"""audi""","""std""","""four""","""sedan""","""4wd""","""front""","""ohc""","""five""","""mpfi""",false,true
"""audi""","""std""","""four""","""sedan""","""fwd""","""front""","""ohc""","""five""","""mpfi""",false,true
"""audi""","""turbo""","""four""","""sedan""","""fwd""","""front""","""ohc""","""five""","""mpfi""",false,true
"""bmw""","""std""","""two""","""sedan""","""rwd""","""front""","""ohc""","""four""","""mpfi""",false,true
…,…,…,…,…,…,…,…,…,…,…
"""volvo""","""std""","""four""","""sedan""","""rwd""","""front""","""ohc""","""four""","""mpfi""",false,true
"""volvo""","""turbo""","""four""","""sedan""","""rwd""","""front""","""ohc""","""four""","""mpfi""",false,true
"""volvo""","""std""","""four""","""sedan""","""rwd""","""front""","""ohcv""","""six""","""mpfi""",false,true


Se pode facer de xeito xenérico para todas as columnas categóricas

In [34]:
proba = irisDatasetv4.select([pl.col(pl.Categorical)])
expr = [[
        (pl.col(columna) == i ).alias(f'{columna}-{i}')
        for i in proba.get_column(columna).cat.get_categories()
    ] for columna in proba.columns]

# hai unha lista de listas, mellor aplanala
exprt_flat = [item for row in expr for item in row]
print(exprt_flat)
resultado = proba.with_columns(
      exprt_flat
    ).drop(proba.columns)
display(resultado)


[<Expr ['[(col("make")) == (String(alfa…'] at 0x7F177D9E8E80>, <Expr ['[(col("make")) == (String(audi…'] at 0x7F177D9EBBE0>, <Expr ['[(col("make")) == (String(bmw)…'] at 0x7F177D9EABC0>, <Expr ['[(col("make")) == (String(chev…'] at 0x7F177D9EA260>, <Expr ['[(col("make")) == (String(dodg…'] at 0x7F177D9E8940>, <Expr ['[(col("make")) == (String(hond…'] at 0x7F177D9E9720>, <Expr ['[(col("make")) == (String(isuz…'] at 0x7F177D9EB9A0>, <Expr ['[(col("make")) == (String(jagu…'] at 0x7F177D9EB100>, <Expr ['[(col("make")) == (String(mazd…'] at 0x7F177D9ABD90>, <Expr ['[(col("make")) == (String(merc…'] at 0x7F177D9AB4F0>, <Expr ['[(col("make")) == (String(merc…'] at 0x7F177D9AB940>, <Expr ['[(col("make")) == (String(mits…'] at 0x7F177D9AB2B0>, <Expr ['[(col("make")) == (String(niss…'] at 0x7F177D9ABEB0>, <Expr ['[(col("make")) == (String(peug…'] at 0x7F177D9ABD00>, <Expr ['[(col("make")) == (String(plym…'] at 0x7F177D9ABD30>, <Expr ['[(col("make")) == (String(pors…'] at 0x7F177D9ABD60>, <Expr [

make-alfa-romero,make-audi,make-bmw,make-chevrolet,make-dodge,make-honda,make-isuzu,make-jaguar,make-mazda,make-mercedes-benz,make-mercury,make-mitsubishi,make-nissan,make-peugot,make-plymouth,make-porsche,make-renault,make-saab,make-subaru,make-toyota,make-volkswagen,make-volvo,fuel-type-gas,fuel-type-diesel,aspiration-std,aspiration-turbo,num-of-doors-two,num-of-doors-four,body-style-convertible,body-style-hatchback,body-style-sedan,body-style-wagon,body-style-hardtop,drive-wheels-rwd,drive-wheels-fwd,drive-wheels-4wd,engine-location-front,engine-location-rear,engine-type-dohc,engine-type-ohcv,engine-type-ohc,engine-type-l,engine-type-rotor,engine-type-ohcf,engine-type-dohcv,num-of-cylinders-four,num-of-cylinders-six,num-of-cylinders-five,num-of-cylinders-three,num-of-cylinders-twelve,num-of-cylinders-two,num-of-cylinders-eight,fuel-system-mpfi,fuel-system-2bbl,fuel-system-mfi,fuel-system-1bbl,fuel-system-spfi,fuel-system-4bbl,fuel-system-idi,fuel-system-spdi
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,true,false,false,true,false,false,true,false,false,false,true,false,true,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,false
false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,true,false,false,true,false,false,true,false,false,false,false,true,true,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,false
false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,true,false,false,true,false,false,true,false,false,false,true,false,true,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,false
false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,true,false,true,false,false,true,false,false,false,true,false,true,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,false
false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,true,false,true,false,false,false,true,false,false,true,false,false,true,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,true,false,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,true,false,false,true,false,true,false,false,true,false,false,true,false,false,true,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,true,false,true,false,f

# Para poder exportalo

O problema con este sistema, é que logo habería que poder repertir este proceso posteriormente en inferencia, sen volver a acceder ao conxunto completo de datos.

A idea parte por almacenar os datos usados para facer a expresión... por exemplo nun JSON.



In [35]:
proba = irisDatasetv4.select([pl.col(pl.Categorical)])
print(proba.columns)
dummies = [[ {"nome": columna, 'valor': i}
        for i in proba.get_column(columna).cat.get_categories()
    ] for columna in proba.columns]
dummies_flat = [item for row in dummies for item in row]
display(dummies_flat)
print(len(dummies_flat))
# agora xa se pode gardar o diccionario como se queira


['make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'engine-type', 'num-of-cylinders', 'fuel-system']


[{'nome': 'make', 'valor': 'alfa-romero'},
 {'nome': 'make', 'valor': 'audi'},
 {'nome': 'make', 'valor': 'bmw'},
 {'nome': 'make', 'valor': 'chevrolet'},
 {'nome': 'make', 'valor': 'dodge'},
 {'nome': 'make', 'valor': 'honda'},
 {'nome': 'make', 'valor': 'isuzu'},
 {'nome': 'make', 'valor': 'jaguar'},
 {'nome': 'make', 'valor': 'mazda'},
 {'nome': 'make', 'valor': 'mercedes-benz'},
 {'nome': 'make', 'valor': 'mercury'},
 {'nome': 'make', 'valor': 'mitsubishi'},
 {'nome': 'make', 'valor': 'nissan'},
 {'nome': 'make', 'valor': 'peugot'},
 {'nome': 'make', 'valor': 'plymouth'},
 {'nome': 'make', 'valor': 'porsche'},
 {'nome': 'make', 'valor': 'renault'},
 {'nome': 'make', 'valor': 'saab'},
 {'nome': 'make', 'valor': 'subaru'},
 {'nome': 'make', 'valor': 'toyota'},
 {'nome': 'make', 'valor': 'volkswagen'},
 {'nome': 'make', 'valor': 'volvo'},
 {'nome': 'fuel-type', 'valor': 'gas'},
 {'nome': 'fuel-type', 'valor': 'diesel'},
 {'nome': 'aspiration', 'valor': 'std'},
 {'nome': 'aspiration', 

60


Agora ese *dummies_flat* pode ser serializado con pickle ou con json e cargarse noutro lado.

Falta xerar a expresión a partires do contido do dict.

In [36]:
# a partires da lista de diccionarios creada anteriormente, que se pode salvar e cargar
novo_expr = [(pl.col(item['nome']) == item["valor"] ).alias(f'{item["nome"]}-{item["valor"]}') for item in dummies_flat]
print(novo_expr)

[<Expr ['[(col("make")) == (String(alfa…'] at 0x7F177D9E98D0>, <Expr ['[(col("make")) == (String(audi…'] at 0x7F177D9EB7F0>, <Expr ['[(col("make")) == (String(bmw)…'] at 0x7F177D9EA200>, <Expr ['[(col("make")) == (String(chev…'] at 0x7F177D9EADA0>, <Expr ['[(col("make")) == (String(dodg…'] at 0x7F177D9EAB60>, <Expr ['[(col("make")) == (String(hond…'] at 0x7F177D9EB790>, <Expr ['[(col("make")) == (String(isuz…'] at 0x7F177D9E9F30>, <Expr ['[(col("make")) == (String(jagu…'] at 0x7F177D9E8FD0>, <Expr ['[(col("make")) == (String(mazd…'] at 0x7F177D9E9660>, <Expr ['[(col("make")) == (String(merc…'] at 0x7F177D9EB730>, <Expr ['[(col("make")) == (String(merc…'] at 0x7F177D9EA110>, <Expr ['[(col("make")) == (String(mits…'] at 0x7F177D9EA740>, <Expr ['[(col("make")) == (String(niss…'] at 0x7F177D9E9D20>, <Expr ['[(col("make")) == (String(peug…'] at 0x7F177D9E83D0>, <Expr ['[(col("make")) == (String(plym…'] at 0x7F177D9EB010>, <Expr ['[(col("make")) == (String(pors…'] at 0x7F177D9E82E0>, <Expr [

In [37]:
from torch.utils.data import Dataset

class AutoDataset(Dataset):
  def __init__(self, src_file, root_dir, transform=None, expr_dummies = None):
    # data like: 5.0, 3.5, 1.3, 0.3, 0
    self.transform = transform
    self.expr_dummies = expr_dummies
    self.dataSet = pl.scan_csv(src_file, dtypes={
            "column_1": pl.Int8,
            "column_2": pl.Float32,
            "column_3": pl.Categorical,
            "column_4": pl.Categorical,
            "column_5": pl.Categorical,
            "column_6": pl.Categorical,
            "column_7": pl.Categorical,
            "column_8": pl.Categorical,
            "column_9": pl.Categorical,
            "column_10": pl.Float32,
            "column_11": pl.Float32,
            "column_12": pl.Float32,
            "column_13": pl.Float32,
            "column_14": pl.Float32,
            "column_15": pl.Categorical,
            "column_16": pl.Categorical,
            "column_17": pl.Float32,
            "column_18": pl.Categorical,
            "column_19": pl.Float32,
            "column_20": pl.Float32,
            "column_21": pl.Float32,
            "column_22": pl.Float32,
            "column_23": pl.Float32,
            "column_24": pl.Float32,
            "column_25": pl.Float32,
            "column_26": pl.Float32
        },has_header=False,null_values=['?']).drop_nulls().rename({
            "column_1": "symboling",
            "column_2": "normalized-losses",
            "column_3": "make",
            "column_4": "fuel-type",
            "column_5": "aspiration",
            "column_6": "num-of-doors",
            "column_7": "body-style",
            "column_8": "drive-wheels",
            "column_9": "engine-location",
            "column_10": "wheel-base",
            "column_11": "length",
            "column_12": "width",
            "column_13": "height",
            "column_14": "curb-weight",
            "column_15": "engine-type",
            "column_16": "num-of-cylinders",
            "column_17": "engine-size",
            "column_18": "fuel-system",
            "column_19": "bore",
            "column_20": "stroke",
            "column_21": "compression-ratio",
            "column_22": "horsepower",
            "column_23": "peak-rpm",
            "column_24": "city-mpg",
            "column_25": "highway-mpg",
            "column_26": "price"
        }).with_row_index("id")

  def __len__(self):
    return self.dataSet.select(pl.len()).collect().item()

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    else:
      idx = [idx]
    # en datos vou a ter as filas que se pediron a __getitem__
    seccion = self.dataSet.filter(pl.col("id").is_in(idx)).drop("id")
    datos = seccion.collect()

    # en datosNumericos teño as columnas con valores numericos, se van a escalar    
    datosNumericos = datos.select([pl.col(pl.Int8),pl.col(pl.Float32)])
    predsA =  self.transform.transform(torch.tensor(datosNumericos.to_numpy()).squeeze()) 
    print(predsA)
    # isto ten 15 elementos scalados. os 14 primeiros son datos de entrada, o 15 e a saída desexada

    # en predsC vou a ter as columnas feitas dummies
    predsC = datos.select([pl.col(pl.Categorical)]).with_columns(
      self.expr_dummies
    ).drop(datos.select([pl.col(pl.Categorical)]).columns)

    # se transforma o dataset cos dummies en un tensor
    tensorB = torch.tensor(predsC.to_numpy().astype(np.int32)).squeeze()
    
    # dim=-1 para que se poñan un tensor ao lado do outro, senon se apilarían en vertical e non coinciden os tamaños
    entrada = torch.cat((predsA[:15], tensorB),dim=-1)
    # as veces pode ser que predsA sexa un vector dunha dimensión, cando o parametro idx é un numero
    # ou pode ser que sexa un unha matriz, se idx é un vector (se piden varios samples para batches)
    # en todo caso se quere o elemento 16, das filas que sexan
    sample = (entrada,predsA[15:] if predsA.dim() == 1 else predsA[:,15:].flatten())
    return sample

       

dataset = AutoDataset("imports-85.data",".",transform=scaler,expr_dummies=novo_expr)
print(dataset[0])
print(dataset[10])
print(dataset[torch.tensor([0,10,100])])


tensor([ 1.0596,  1.2024,  0.2972,  0.3633,  0.3042,  0.1766, -0.2576, -0.3357,
        -0.4119,  0.5549, -0.0414,  0.2006,  0.8291, -0.4136, -0.3223,  0.4261],
       dtype=torch.float64)
(tensor([ 1.0596,  1.2024,  0.2972,  0.3633,  0.3042,  0.1766, -0.2576, -0.3357,
        -0.4119,  0.5549, -0.0414,  0.2006,  0.8291, -0.4136, -0.3223,  0.0000,
         1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,
         0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], dtype=torch.float64), tensor([0.4261], dtype=torch.flo